# Market Risk Early Warning System - Sentiment Analysis

This notebook demonstrates the sentiment analysis component of MEWS, including:
- News data collection from GNews API
- VADER sentiment analysis
- Sentiment aggregation and scoring
- Integration with risk predictions
- Real-time sentiment monitoring

In [ ]:
# Import required libraries
import sys
import os
sys.path.append('..')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Sentiment analysis libraries
import requests
import json
from datetime import datetime, timedelta
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import nltk

# Ensure NLTK data is downloaded
try:
    nltk.data.find('vader_lexicon')
except LookupError:
    nltk.download('vader_lexicon')

# Configuration
import warnings
warnings.filterwarnings('ignore')

# Set style
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print("Libraries imported successfully!")
print("VADER sentiment analyzer ready!")

## GNews API Integration

First, let's set up the GNews API integration for fetching financial news.

In [ ]:
class NewsCollector:
    """Collect and analyze financial news sentiment"""
    
    def __init__(self, api_key=None):
        self.api_key = api_key or "0903e69179300b9e3117cdc721c14366"  # Default API key
        self.base_url = "https://gnews.io/api/v4/search"
        self.analyzer = SentimentIntensityAnalyzer()
        
    def fetch_gnews_sentiment(self, symbols, days=7, max_articles=50):
        """Fetch news sentiment from GNews API"""
        
        if not self.api_key:
            print("❌ No API key provided")
            return None
        
        all_news = []
        
        print(f"🔍 Fetching news for {len(symbols)} symbols over {days} days...")
        
        for symbol in symbols:
            try:
                # Calculate date range
                end_date = datetime.now()
                start_date = end_date - timedelta(days=days)
                
                # API parameters
                params = {
                    'q': f'{symbol} stock market financial',
                    'token': self.api_key,
                    'lang': 'en',
                    'country': 'us',
                    'max': min(max_articles, 10),  # GNews API limits
                    'from': start_date.strftime('%Y-%m-%dT%H:%M:%SZ'),
                    'to': end_date.strftime('%Y-%m-%dT%H:%M:%SZ')
                }
                
                # Make API request
                response = requests.get(self.base_url, params=params, timeout=10)
                
                if response.status_code == 200:
                    data = response.json()
                    articles = data.get('articles', [])
                    
                    print(f"  ✅ {symbol}: {len(articles)} articles found")
                    
                    for article in articles:
                        news_item = {
                            'symbol': symbol,
                            'title': article.get('title', ''),
                            'description': article.get('description', ''),
                            'content': article.get('content', ''),
                            'published_date': article.get('publishedAt', ''),
                            'source': article.get('source', {}).get('name', ''),
                            'url': article.get('url', '')
                        }
                        all_news.append(news_item)
                else:
                    print(f"  ❌ {symbol}: API request failed ({response.status_code})")
                    
            except Exception as e:
                print(f"  ❌ {symbol}: Error - {str(e)}")
        
        print(f"📰 Total articles collected: {len(all_news)}")
        return pd.DataFrame(all_news) if all_news else None
    
    def analyze_sentiment(self, text):
        """Analyze sentiment of text using VADER"""
        if not text or pd.isna(text):
            return {'compound': 0.0, 'pos': 0.0, 'neu': 0.0, 'neg': 0.0}
        
        scores = self.analyzer.polarity_scores(str(text))
        return scores
    
    def process_news_sentiment(self, news_df):
        """Process news dataframe and add sentiment scores"""
        if news_df is None or news_df.empty:
            return None
        
        print("🔄 Analyzing sentiment for all articles...")
        
        # Combine title and description for sentiment analysis
        news_df['combined_text'] = (
            news_df['title'].fillna('') + ' ' + 
            news_df['description'].fillna('')
        )
        
        # Analyze sentiment for each article
        sentiment_scores = []
        for text in news_df['combined_text']:
            scores = self.analyze_sentiment(text)
            sentiment_scores.append(scores)
        
        # Convert to DataFrame and merge
        sentiment_df = pd.DataFrame(sentiment_scores)
        result_df = pd.concat([news_df, sentiment_df], axis=1)
        
        # Add sentiment interpretation
        result_df['sentiment_label'] = result_df['compound'].apply(
            lambda x: 'Positive' if x > 0.05 else ('Negative' if x < -0.05 else 'Neutral')
        )
        
        print(f"✅ Sentiment analysis completed for {len(result_df)} articles")
        
        return result_df

# Initialize news collector
news_collector = NewsCollector()
print("📰 News Collector initialized!")

## Fetch and Analyze Recent News

Let's fetch recent news for major stocks and analyze their sentiment.

In [ ]:
# Define stocks to analyze
symbols = ['AAPL', 'MSFT', 'GOOGL', 'TSLA', 'NVDA', 'JPM', 'BAC']

# Fetch news data
print("🔍 Starting news collection...")
news_df = news_collector.fetch_gnews_sentiment(symbols, days=7, max_articles=50)

if news_df is not None and not news_df.empty:
    print(f"📊 News data shape: {news_df.shape}")
    print(f"📈 Symbols covered: {news_df['symbol'].unique()}")
    print(f"📅 Date range: {news_df['published_date'].min()} to {news_df['published_date'].max()}")
    
    # Display sample articles
    print("\n📰 Sample Articles:")
    print("-" * 80)
    for idx, row in news_df.head(3).iterrows():
        print(f"Symbol: {row['symbol']}")
        print(f"Title: {row['title']}")
        print(f"Source: {row['source']}")
        print(f"Date: {row['published_date']}")
        print("-" * 80)
else:
    print("❌ No news data available. Using sample data for demonstration.")
    
    # Create sample news data for demonstration
    sample_news = [
        {
            'symbol': 'AAPL',
            'title': 'Apple reports strong quarterly earnings with iPhone sales growth',
            'description': 'Apple Inc. exceeded expectations with robust iPhone sales and services revenue growth',
            'published_date': '2025-09-29T10:30:00Z',
            'source': 'Financial News'
        },
        {
            'symbol': 'TSLA',
            'title': 'Tesla faces production challenges amid supply chain issues',
            'description': 'Electric vehicle manufacturer Tesla encounters manufacturing bottlenecks',
            'published_date': '2025-09-29T08:15:00Z',
            'source': 'Market Watch'
        },
        {
            'symbol': 'MSFT',
            'title': 'Microsoft Azure cloud services show continued expansion',
            'description': 'Microsoft Corporation reports significant growth in cloud computing revenue',
            'published_date': '2025-09-29T14:20:00Z',
            'source': 'Tech Daily'
        }
    ]
    
    news_df = pd.DataFrame(sample_news)
    print(f"📊 Using sample data: {news_df.shape}")

In [ ]:
# Process sentiment analysis
if news_df is not None:
    sentiment_df = news_collector.process_news_sentiment(news_df)
    
    if sentiment_df is not None:
        print("📊 Sentiment Analysis Results:")
        print("=" * 50)
        
        # Display sentiment statistics
        print(f"Total articles analyzed: {len(sentiment_df)}")
        print(f"Sentiment distribution:")
        print(sentiment_df['sentiment_label'].value_counts())
        print(f"\nAverage sentiment scores:")
        print(f"  Compound: {sentiment_df['compound'].mean():.3f}")
        print(f"  Positive: {sentiment_df['pos'].mean():.3f}")
        print(f"  Negative: {sentiment_df['neg'].mean():.3f}")
        print(f"  Neutral: {sentiment_df['neu'].mean():.3f}")
        
        # Show sample results
        print(f"\n📰 Sample Sentiment Results:")
        print("-" * 80)
        for idx, row in sentiment_df.head(3).iterrows():
            print(f"Symbol: {row['symbol']}")
            print(f"Title: {row['title'][:60]}...")
            print(f"Sentiment: {row['sentiment_label']} (Score: {row['compound']:.3f})")
            print(f"Positive: {row['pos']:.3f}, Negative: {row['neg']:.3f}, Neutral: {row['neu']:.3f}")
            print("-" * 80)
    else:
        print("❌ Sentiment analysis failed")
else:
    print("❌ No news data to analyze")

## Sentiment Visualization and Analysis

Let's create comprehensive visualizations of the sentiment analysis results.

In [ ]:
if 'sentiment_df' in locals() and sentiment_df is not None:
    
    # Create comprehensive sentiment visualizations
    fig = make_subplots(
        rows=2, cols=2,
        subplot_titles=(
            'Sentiment Distribution by Stock',
            'Sentiment Score Distribution',
            'Average Sentiment by Symbol',
            'Sentiment Timeline'
        ),
        specs=[[{"type": "bar"}, {"type": "histogram"}],
               [{"type": "bar"}, {"type": "scatter"}]]
    )
    
    # 1. Sentiment distribution by stock
    sentiment_by_stock = sentiment_df.groupby(['symbol', 'sentiment_label']).size().reset_index(name='count')
    
    for sentiment in ['Positive', 'Negative', 'Neutral']:
        data = sentiment_by_stock[sentiment_by_stock['sentiment_label'] == sentiment]
        fig.add_trace(
            go.Bar(
                name=sentiment,
                x=data['symbol'],
                y=data['count'],
                text=data['count'],
                textposition='auto'
            ),
            row=1, col=1
        )
    
    # 2. Sentiment score distribution
    fig.add_trace(
        go.Histogram(
            x=sentiment_df['compound'],
            nbinsx=20,
            name='Compound Score',
            marker_color='lightblue',
            showlegend=False
        ),
        row=1, col=2
    )
    
    # 3. Average sentiment by symbol
    avg_sentiment = sentiment_df.groupby('symbol')['compound'].mean().reset_index()
    avg_sentiment = avg_sentiment.sort_values('compound', ascending=True)
    
    colors = ['red' if x < -0.05 else 'green' if x > 0.05 else 'gray' for x in avg_sentiment['compound']]
    
    fig.add_trace(
        go.Bar(
            x=avg_sentiment['compound'],
            y=avg_sentiment['symbol'],
            orientation='h',
            marker_color=colors,
            text=[f'{x:.3f}' for x in avg_sentiment['compound']],
            textposition='auto',
            showlegend=False
        ),
        row=2, col=1
    )
    
    # 4. Sentiment timeline (if we have dates)
    if 'published_date' in sentiment_df.columns:
        # Convert dates and create timeline
        sentiment_df['date'] = pd.to_datetime(sentiment_df['published_date'])
        daily_sentiment = sentiment_df.groupby([sentiment_df['date'].dt.date, 'symbol'])['compound'].mean().reset_index()
        
        for symbol in sentiment_df['symbol'].unique():
            symbol_data = daily_sentiment[daily_sentiment['symbol'] == symbol]
            fig.add_trace(
                go.Scatter(
                    x=symbol_data['date'],
                    y=symbol_data['compound'],
                    mode='lines+markers',
                    name=symbol,
                    showlegend=True
                ),
                row=2, col=2
            )
    
    # Update layout
    fig.update_layout(
        height=800,
        title_text="Financial News Sentiment Analysis Dashboard",
        showlegend=True
    )
    
    # Update axes
    fig.update_xaxes(title_text="Symbols", row=1, col=1)
    fig.update_yaxes(title_text="Article Count", row=1, col=1)
    fig.update_xaxes(title_text="Compound Sentiment Score", row=1, col=2)
    fig.update_yaxes(title_text="Frequency", row=1, col=2)
    fig.update_xaxes(title_text="Average Sentiment Score", row=2, col=1)
    fig.update_yaxes(title_text="Symbols", row=2, col=1)
    fig.update_xaxes(title_text="Date", row=2, col=2)
    fig.update_yaxes(title_text="Sentiment Score", row=2, col=2)
    
    fig.show()
    
    print("📊 Sentiment visualizations created!")
else:
    print("❌ No sentiment data available for visualization")

## Sentiment Aggregation and Scoring

Let's create aggregated sentiment scores that can be used in the risk prediction models.

In [ ]:
def aggregate_sentiment_scores(sentiment_df, method='weighted_average'):
    """Aggregate sentiment scores for risk prediction integration"""
    
    if sentiment_df is None or sentiment_df.empty:
        return None
    
    print(f"🔄 Aggregating sentiment scores using {method} method...")
    
    # Group by symbol and calculate various aggregations
    aggregations = {
        'compound': ['mean', 'std', 'min', 'max', 'count'],
        'pos': ['mean', 'sum'],
        'neg': ['mean', 'sum'],
        'neu': ['mean']
    }
    
    sentiment_agg = sentiment_df.groupby('symbol').agg(aggregations).round(4)
    sentiment_agg.columns = ['_'.join(col).strip() for col in sentiment_agg.columns]
    sentiment_agg = sentiment_agg.reset_index()
    
    # Calculate weighted sentiment score (considering article count and recency)
    sentiment_agg['weighted_sentiment'] = (
        sentiment_agg['compound_mean'] * 
        np.log1p(sentiment_agg['compound_count']) * 0.1  # Weight by article count
    )
    
    # Create sentiment risk indicators
    sentiment_agg['sentiment_risk'] = sentiment_agg['compound_mean'].apply(
        lambda x: 1 if x < -0.1 else (0.5 if -0.1 <= x < 0.1 else 0)
    )
    
    # Calculate sentiment volatility (how much sentiment varies)
    sentiment_agg['sentiment_volatility'] = sentiment_agg['compound_std'].fillna(0)
    
    # Create composite sentiment score
    sentiment_agg['composite_sentiment'] = (
        0.6 * sentiment_agg['compound_mean'] +
        0.2 * sentiment_agg['pos_mean'] -
        0.2 * sentiment_agg['neg_mean']
    )
    
    print(f"✅ Sentiment aggregation completed for {len(sentiment_agg)} symbols")
    
    return sentiment_agg

def interpret_sentiment_for_users(sentiment_score):
    """Create user-friendly interpretation of sentiment scores"""
    
    interpretations = []
    
    if sentiment_score > 0.3:
        interpretations.append("📈 **Strongly Positive**: News sentiment is very bullish, indicating optimism about the stock's prospects.")
    elif sentiment_score > 0.1:
        interpretations.append("📊 **Moderately Positive**: News sentiment leans positive, suggesting favorable market conditions.")
    elif sentiment_score > -0.1:
        interpretations.append("😐 **Neutral**: News sentiment is balanced, with mixed positive and negative coverage.")
    elif sentiment_score > -0.3:
        interpretations.append("📉 **Moderately Negative**: News sentiment leans negative, indicating some concerns or challenges.")
    else:
        interpretations.append("🚨 **Strongly Negative**: News sentiment is very bearish, suggesting significant negative developments.")
    
    # Add risk implications
    if sentiment_score < -0.2:
        interpretations.append("⚠️ **Risk Implication**: Negative sentiment may indicate increased market risk and potential price volatility.")
    elif sentiment_score > 0.2:
        interpretations.append("✅ **Risk Implication**: Positive sentiment suggests lower immediate risk and potential upward momentum.")
    else:
        interpretations.append("🔄 **Risk Implication**: Neutral sentiment indicates stable conditions with no immediate sentiment-driven risk.")
    
    return " ".join(interpretations)

# Aggregate sentiment scores
if 'sentiment_df' in locals() and sentiment_df is not None:
    sentiment_aggregated = aggregate_sentiment_scores(sentiment_df)
    
    if sentiment_aggregated is not None:
        print("📊 AGGREGATED SENTIMENT RESULTS:")
        print("=" * 60)
        
        # Display results
        for idx, row in sentiment_aggregated.iterrows():
            symbol = row['symbol']
            avg_sentiment = row['compound_mean']
            risk_score = row['sentiment_risk']
            
            print(f"\n{symbol}:")
            print(f"  Average Sentiment: {avg_sentiment:.3f}")
            print(f"  Risk Score: {risk_score:.1f}")
            print(f"  Article Count: {int(row['compound_count'])}")
            print(f"  Interpretation: {interpret_sentiment_for_users(avg_sentiment)}")
        
        # Save aggregated results
        output_file = "../data/sentiment_aggregated.csv"
        sentiment_aggregated.to_csv(output_file, index=False)
        print(f"\n💾 Sentiment data saved to: {output_file}")
    else:
        print("❌ Sentiment aggregation failed")
else:
    print("❌ No sentiment data to aggregate")

## Real-time Sentiment Monitoring Function

Let's create a function that can be used for real-time sentiment monitoring in production.

In [ ]:
def monitor_sentiment_realtime(symbols, api_key, lookback_hours=24):
    """Real-time sentiment monitoring function for production use"""
    
    print(f"🔍 Starting real-time sentiment monitoring for {len(symbols)} symbols...")
    print(f"Lookback period: {lookback_hours} hours")
    
    collector = NewsCollector(api_key)
    
    # Fetch recent news
    lookback_days = max(1, lookback_hours // 24)
    news_df = collector.fetch_gnews_sentiment(symbols, days=lookback_days)
    
    if news_df is None or news_df.empty:
        print("⚠️ No recent news found")
        return None
    
    # Filter to recent hours if needed
    if lookback_hours < 24:
        cutoff_time = datetime.now() - timedelta(hours=lookback_hours)
        news_df['published_date'] = pd.to_datetime(news_df['published_date'])
        news_df = news_df[news_df['published_date'] >= cutoff_time]
    
    # Analyze sentiment
    sentiment_df = collector.process_news_sentiment(news_df)
    
    if sentiment_df is None:
        print("❌ Sentiment analysis failed")
        return None
    
    # Create summary report
    summary = {
        'timestamp': datetime.now().isoformat(),
        'symbols_analyzed': symbols,
        'total_articles': len(sentiment_df),
        'sentiment_summary': {}
    }
    
    for symbol in symbols:
        symbol_data = sentiment_df[sentiment_df['symbol'] == symbol]
        
        if not symbol_data.empty:
            avg_sentiment = symbol_data['compound'].mean()
            article_count = len(symbol_data)
            
            summary['sentiment_summary'][symbol] = {
                'average_sentiment': round(avg_sentiment, 3),
                'article_count': article_count,
                'sentiment_label': 'Positive' if avg_sentiment > 0.05 else ('Negative' if avg_sentiment < -0.05 else 'Neutral'),
                'risk_indicator': 'High' if avg_sentiment < -0.2 else ('Low' if avg_sentiment > 0.2 else 'Medium'),
                'interpretation': interpret_sentiment_for_users(avg_sentiment)
            }
        else:
            summary['sentiment_summary'][symbol] = {
                'average_sentiment': 0.0,
                'article_count': 0,
                'sentiment_label': 'No Data',
                'risk_indicator': 'Unknown',
                'interpretation': 'No recent news available for sentiment analysis.'
            }
    
    print("✅ Real-time sentiment monitoring completed!")
    return summary

# Example of real-time monitoring
if 'news_collector' in locals():
    print("\n🔄 Testing real-time sentiment monitoring...")
    
    test_symbols = ['AAPL', 'TSLA', 'MSFT']
    monitoring_result = monitor_sentiment_realtime(
        symbols=test_symbols,
        api_key="0903e69179300b9e3117cdc721c14366",
        lookback_hours=24
    )
    
    if monitoring_result:
        print("\n📊 REAL-TIME SENTIMENT MONITORING RESULTS:")
        print("=" * 60)
        print(f"Timestamp: {monitoring_result['timestamp']}")
        print(f"Total Articles: {monitoring_result['total_articles']}")
        
        for symbol, data in monitoring_result['sentiment_summary'].items():
            print(f"\n{symbol}:")
            print(f"  Sentiment: {data['sentiment_label']} ({data['average_sentiment']})")
            print(f"  Risk Level: {data['risk_indicator']}")
            print(f"  Articles: {data['article_count']}")
            print(f"  Analysis: {data['interpretation'][:100]}...")
    else:
        print("❌ Real-time monitoring test failed")

## Integration with Risk Prediction Models

Let's show how sentiment data integrates with the ML risk prediction models.

In [ ]:
def create_sentiment_features_for_ml(sentiment_aggregated):
    """Create sentiment features that can be used in ML risk models"""
    
    if sentiment_aggregated is None:
        return None
    
    print("🔄 Creating sentiment features for ML integration...")
    
    # Create feature set for ML models
    ml_features = sentiment_aggregated[['symbol']].copy()
    
    # Direct sentiment features
    ml_features['sentiment_compound'] = sentiment_aggregated['compound_mean']
    ml_features['sentiment_volatility'] = sentiment_aggregated['sentiment_volatility']
    ml_features['sentiment_positive'] = sentiment_aggregated['pos_mean']
    ml_features['sentiment_negative'] = sentiment_aggregated['neg_mean']
    
    # Derived features
    ml_features['sentiment_strength'] = abs(sentiment_aggregated['compound_mean'])
    ml_features['sentiment_bias'] = (
        sentiment_aggregated['pos_mean'] - sentiment_aggregated['neg_mean']
    )
    ml_features['news_coverage'] = np.log1p(sentiment_aggregated['compound_count'])
    
    # Risk indicators
    ml_features['sentiment_risk_binary'] = (sentiment_aggregated['compound_mean'] < -0.1).astype(int)
    ml_features['sentiment_opportunity'] = (sentiment_aggregated['compound_mean'] > 0.2).astype(int)
    
    # Sentiment momentum (simplified - would need time series data for real momentum)
    ml_features['sentiment_momentum'] = sentiment_aggregated['compound_mean'] * sentiment_aggregated['compound_count']
    
    print(f"✅ Created {len(ml_features.columns)-1} sentiment features for ML integration")
    
    return ml_features

def combine_sentiment_with_market_data(sentiment_features, market_data):
    """Combine sentiment features with market data for comprehensive risk analysis"""
    
    if sentiment_features is None or market_data is None:
        print("❌ Cannot combine data - missing sentiment or market data")
        return None
    
    print("🔄 Combining sentiment and market data...")
    
    # Merge on symbol
    combined = market_data.merge(sentiment_features, on='symbol', how='left')
    
    # Fill missing sentiment data with neutral values
    sentiment_cols = [col for col in sentiment_features.columns if col != 'symbol']
    combined[sentiment_cols] = combined[sentiment_cols].fillna(0)
    
    # Create interaction features
    if 'Close' in combined.columns:
        combined['sentiment_price_interaction'] = combined['sentiment_compound'] * combined['Close']
    
    if 'Volume' in combined.columns:
        combined['sentiment_volume_interaction'] = combined['sentiment_compound'] * combined['Volume']
    
    print(f"✅ Combined dataset created: {combined.shape}")
    
    return combined

# Create sentiment features for ML
if 'sentiment_aggregated' in locals() and sentiment_aggregated is not None:
    
    sentiment_features = create_sentiment_features_for_ml(sentiment_aggregated)
    
    if sentiment_features is not None:
        print("\n📊 SENTIMENT FEATURES FOR ML:")
        print("=" * 50)
        print(f"Features created: {list(sentiment_features.columns)}")
        print(f"Sample data:")
        print(sentiment_features.head())
        
        # Save sentiment features
        features_file = "../data/sentiment_features_ml.csv"
        sentiment_features.to_csv(features_file, index=False)
        print(f"\n💾 Sentiment features saved to: {features_file}")
        
        # Example of how this would integrate with market data
        print("\n🔗 INTEGRATION EXAMPLE:")
        print("This sentiment data can be merged with market data using:")
        print("combined_data = market_df.merge(sentiment_features, on='symbol', how='left')")
        print("Then use in ML models as additional features for risk prediction!")
    
else:
    print("❌ No sentiment data available for ML feature creation")

## Summary and Production Usage

This notebook demonstrated the complete sentiment analysis pipeline for MEWS:

### 🎯 **What we accomplished:**
- ✅ GNews API integration for financial news collection
- ✅ VADER sentiment analysis implementation
- ✅ Comprehensive sentiment visualization and analysis
- ✅ Sentiment aggregation and scoring for risk assessment
- ✅ Real-time sentiment monitoring capabilities
- ✅ ML feature engineering from sentiment data
- ✅ Integration framework with risk prediction models

### 📊 **Key Features:**
- **Real-time News**: Fetch up-to-date financial news from multiple sources
- **Advanced Sentiment**: VADER analysis with custom interpretations
- **Risk Integration**: Sentiment-based risk indicators
- **User-Friendly**: Plain English explanations of sentiment analysis
- **Production Ready**: Scalable functions for live deployment

### 🚀 **Production Usage:**
The functions in this notebook are used in:
1. **Streamlit Dashboard**: Real-time sentiment display
2. **ML Risk Models**: Sentiment as additional features
3. **Risk Timeline**: Sentiment-driven risk assessment
4. **Automated Monitoring**: Scheduled sentiment updates

### 📈 **Business Value:**
- **Early Warning**: Detect sentiment shifts before price movements
- **Risk Management**: Quantify news-driven market risk
- **Decision Support**: Data-driven insights for investment decisions
- **Market Intelligence**: Comprehensive view of market sentiment

The sentiment analysis system is now ready for production deployment! 🎉